In [1]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd

%matplotlib inline

In [2]:
df1 = pd.read_csv("../data/shrek1_dialog_w_timestamps.csv", index_col="Unnamed: 0")
df2 = pd.read_csv("../data/shrek2_dialog_w_timestamps.csv", index_col="Unnamed: 0")
df3 = pd.read_csv("../data/shrek3_dialog_w_timestamps.csv", index_col="Unnamed: 0")
df = pd.concat([df1, df2, df3])

In [3]:
df['dialog_clean'] = df.dialog.str.replace(r'[,\'".?!~\t]', '')
df['dialog_clean'] = df.dialog_clean.str.replace('-', ' ')
df['dialog_clean'] = df.dialog_clean.str.strip()
df = df[~df.dialog.isnull()]

df['word_count'] = df.dialog_clean.str.split().apply(len)

word_counts = df.groupby('name').word_count.sum()
top_chars = list(df.groupby('name').word_count.sum().drop('SOUND FX').sort_values(ascending=False)[:20].index)
df = df[df.name.isin(top_chars)]

In [4]:
df1 = df[df.name != df.name.shift(-1)]

In [5]:
converse = pd.DataFrame({"name1": df1.name.values, "name2": df1.name.shift(-1).values})
converse = converse[(converse.name1!="MUSIC")&(converse.name2!="MUSIC")]

In [6]:
replacements = {'CINDERALLA': 'CINDERELLA', 'CAPTAIN HOOKS': 'CAPTAIN HOOK',
                'PRINCE CHARING': 'PRINCE CHARMING', 'PRINE CHARMING': 'PRINCE CHARMING',
               'BIG BAD WOLF': 'WOLFIE', 'MOUSE': 'BLIND MOUSE', 'AUDIENCE': 'CROWD',
               'GINGERBREAD MAN': 'GINGY', 'KING': 'KING HAROLD', 'SHEK': 'SHREK'}
todrop = ['OGRE BABIES', 'INQUISITOR']
converse.loc[:, 'name1'] = converse['name1'].apply(
    lambda x: replacements[x] if x in replacements.keys() else x)

converse.loc[:, 'name2'] = converse['name2'].apply(
    lambda x: replacements[x] if x in replacements.keys() else x)

converse = converse[(~converse['name1'].isin(todrop))&(~converse['name2'].isin(todrop))]

In [7]:
adj_mat = converse.groupby("name1").name2.value_counts().unstack().fillna(0)

In [8]:
gr = nx.from_pandas_adjacency(adj_mat)

In [9]:
jsnx = nx.readwrite.json_graph.cytoscape_data(gr)

In [10]:
json_graph = dict()
json_graph.update({'edges': [i['data'] for i in jsnx['elements']['edges']]})
json_graph.update({'nodes': [{'id': i['data']['id'],
                              'word_count': word_counts[i['data']['id']]} 
                             for i in jsnx['elements']['nodes']]})

In [11]:
[i['id'] for i in json_graph['nodes']]

['DONKEY',
 'SHREK',
 'FAIRY GODMOTHER',
 'RAPUNZEL',
 'SNOW WHITE',
 'PRINCE CHARMING',
 'KING HAROLD',
 'PINOCCHIO',
 'ARTIE',
 'MERLIN',
 'FIONA',
 'GUARD',
 'ROBIN HOOD',
 'PUSS',
 'GINGY',
 'QUEEN',
 'MIRROR',
 'FARQUAAD']

In [12]:
with open("../data/shrek_all_network.json", "w") as f:
    f.write(str(json_graph))